In [ ]:
#r "nuget: System.Reactive"
#r "nuget: System.Linq.Async"
using System.Reactive;
using System.Reactive.Linq;
using System.Reactive.Subjects;

Installed Packages System.Linq.Async, 6.0.1 System.Reactive, 5.0.0

In [ ]:
public static async Task Process<T>(int thread, T value)
{
	await Task.Delay(2000);
	$"[{thread}] {value}".Display();
}

In [ ]:
static async IAsyncEnumerable<int> Range(int start, int count)
{
	for (var i = 0; i < count; ++i)
	{
		await Task.Delay(50);
		yield return start;
		++start;
	}
}

In [ ]:
static class Helper
{
	public static Task ForEachAwaitAsync<T>(/*this*/ IAsyncEnumerable<T> enumerable, Func<int, T, Task> handler, int degreeOfParallelism)
	{
		var subj = new BehaviorSubject<T>(default(T));
		var tasks = new Task[degreeOfParallelism];

		Func<T, int, bool> createFilter(int degree, int id) => (x, i) => (i % degree) == id;
		
		Func<T, Task> createHandler(int id, Func<int, T, Task> basicHandler) => x => basicHandler(id, x);
		
		for (var id = 0; id < degreeOfParallelism; ++id)
			tasks[id] = subj.Skip(1).Where(createFilter(degreeOfParallelism, id)).ToAsyncEnumerable().ForEachAwaitAsync(createHandler(id, handler));
		
		enumerable.ToObservable().Subscribe(subj);
			
		return Task.WhenAll(tasks);
	}
}

In [ ]:
await Helper.ForEachAwaitAsync(
    Range(200, 20).Select(i => i.ToString()),
    (id, x) => Process(id, x),
    5)


[0] 200

[1] 201

[2] 202

[3] 203

[4] 204

...

[0] 215

[1] 216

[2] 217

[3] 218

[4] 219